In [1]:
from utils import load_model, load_config, get_keypoints, draw_keypoints
import cv2
import torch
import numpy as np

In [2]:
config = load_config('./configs/config_YOLO_x.yaml')
model = load_model(config['model'])

In [3]:
image = cv2.imread('./example_images/demo_cropped.jpg')
image = cv2.resize(image, config['dataset']['preprocess']['input_size'])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.astype('float32')
image /= 255.0

image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)
print(image.shape)

torch.Size([1, 3, 640, 640])


In [4]:
model.eval()
out = model(image)
keypoints1 = get_keypoints(out, config)[0]
model.train()
out = model(image)
keypoints2 = get_keypoints(out, config)[0]

In [20]:
image_show = cv2.imread('./example_images/demo_cropped.jpg')
image_show = cv2.resize(image_show, config['dataset']['preprocess']['input_size'])

# image_show1 = draw_keypoints(image_show, keypoints1, np.ones(len(keypoints1)))
image_show2 = draw_keypoints(image_show.copy(), keypoints2, np.ones(len(keypoints2)))

# cv2.imshow('keypoints1', image_show1)
cv2.imshow('keypoints2', image_show2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
loaded_model = torch.load('./weights/yolov8x-pose.pt')
model = loaded_model['model']

In [42]:
rand_input = torch.rand(1, 3, 512, 512)
out = model(rand_input)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.DoubleTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [36]:
print(model.state_dict().keys())

odict_keys(['model.0.conv.weight', 'model.0.bn.weight', 'model.0.bn.bias', 'model.0.bn.running_mean', 'model.0.bn.running_var', 'model.0.bn.num_batches_tracked', 'model.1.conv.weight', 'model.1.bn.weight', 'model.1.bn.bias', 'model.1.bn.running_mean', 'model.1.bn.running_var', 'model.1.bn.num_batches_tracked', 'model.2.cv1.conv.weight', 'model.2.cv1.bn.weight', 'model.2.cv1.bn.bias', 'model.2.cv1.bn.running_mean', 'model.2.cv1.bn.running_var', 'model.2.cv1.bn.num_batches_tracked', 'model.2.cv2.conv.weight', 'model.2.cv2.bn.weight', 'model.2.cv2.bn.bias', 'model.2.cv2.bn.running_mean', 'model.2.cv2.bn.running_var', 'model.2.cv2.bn.num_batches_tracked', 'model.2.m.0.cv1.conv.weight', 'model.2.m.0.cv1.bn.weight', 'model.2.m.0.cv1.bn.bias', 'model.2.m.0.cv1.bn.running_mean', 'model.2.m.0.cv1.bn.running_var', 'model.2.m.0.cv1.bn.num_batches_tracked', 'model.2.m.0.cv2.conv.weight', 'model.2.m.0.cv2.bn.weight', 'model.2.m.0.cv2.bn.bias', 'model.2.m.0.cv2.bn.running_mean', 'model.2.m.0.cv2.bn.

In [40]:
print(dir(model))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_clip_augmented', '_compiled_call_impl', '_descale_pred', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters'

In [43]:
print(model)

PoseModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
 

In [1]:
from models.YOLO.yolo import YOLO, get_yolo_model, get_keypoints_yolo, YoloKeypointLoss
import torch
import cv2
from utils import load_config
from dataset import PoseDataset

model = get_yolo_model('yolov8x-pose', 17)


config = load_config('./configs/config_YOLO_x.yaml')

n = 1
dataset = PoseDataset(config["dataset"], config["dataset"]['val'])

loader = torch.utils.data.DataLoader(dataset, batch_size=n, shuffle=True)
images,targets,keypoints_gts,keypoint_visibilitys = next(iter(loader))

In [2]:
model.eval()
out = model(images)
print(out.shape)
keypoints = get_keypoints_yolo(out)

torch.Size([1, 51, 8400])


In [3]:
loss = YoloKeypointLoss()
loss(out, targets, keypoints_gts, keypoint_visibilitys)

torch.Size([1, 17, 8400])


In [12]:
image_show = images[0].permute(1, 2, 0).numpy()
image_show = cv2.resize(image_show, config['dataset']['preprocess']['input_size'])
image_show = cv2.cvtColor(image_show, cv2.COLOR_RGB2BGR)

for i in range(keypoints.shape[1]):
    x = keypoints[0, i, 0].item()
    y = keypoints[0, i, 1].item()
    conf = keypoints[0, i, 2].item()
    if conf > 0.5:
        cv2.circle(image_show, (int(x), int(y)), 5, (0, 255, 0), -1)

cv2.imshow('keypoints', image_show)
cv2.waitKey(0)
cv2.destroyAllWindows()